In [342]:
from qiskit import *
from qiskit.quantum_info import SparsePauliOp, Statevector
from VQE_CVaR import partition_N
import numpy as np
from qiskit import QuantumRegister, QuantumCircuit

from math import cos, sin, cosh, sinh, atan, exp, pi
from scipy.optimize import minimize

import sys

import copy
import pickle

from qubo_hamiltonian import *

import itertools

In [343]:
def find_light_cone(pairs):
    lightcone_dict = {}
    for index, list in enumerate(pairs):
        for pair in list:
            qi, qj = pair
            relevent_pairs = []  ##  qubit pairs in the previous layer that in the lightcone of the current pair
            if index > 0:
                for pair_layerm1 in pairs[index-1]: ## qubit pairs in the previous layer
                    if (qi in pair_layerm1) or (qj in pair_layerm1):
                        relevent_pairs.append(pair_layerm1)
            lightcone_dict[pair] = relevent_pairs
    return lightcone_dict

In [344]:
def circuit_update_zz(Edge,State, Tauc, N):
    i = min(Edge)
    j = max(Edge)

    zzop = SparsePauliOp.from_sparse_list([('ZZ', [j, i], -sinh(Tauc) )], N)
    zzop += SparsePauliOp.from_list([("I"*N, cosh(Tauc))])
    
    State = State.evolve(zzop)

    # Normalize the state vector manually
    norm = np.linalg.norm(State.data)
    State = State / norm

    # Print the normalized state vector
    # print("Normalized state:", State)

    return State

In [345]:
def circuit_update_theta(Edge, State, paras, N):
    
    j = min(Edge)    #I SWAPPED i AND j TO MATCH WITH YAHUI CIRCUIT
    i = max(Edge)

    zyop = SparsePauliOp.from_sparse_list([('ZY', [j, i], -1j*sin(paras[0]/2))], N)
    zyop += SparsePauliOp.from_list([("I"*N, cos(paras[0]/2))])

    yzop = SparsePauliOp.from_sparse_list([('YZ', [j, i], -1j*sin(paras[1]/2))], N)
    yzop += SparsePauliOp.from_list([("I"*N, cos(paras[1]/2))])

    op = zyop.compose(yzop)
    
    State = State.evolve(op)

    # print('states2', State)

    return State

In [346]:
def circuit_update_theta_Yahui(Edge, Circ, paras):

    qcirc = Circ.copy()
    
    i = min(Edge)
    j = max(Edge)

    ### exp{-i/2 ( params[2]*ZiYj + params[3]*YiZj )}
    qcirc.rx(-np.pi/2, i)
    qcirc.rz(-np.pi/2, j)

    qcirc.cx(i, j)
    qcirc.ry(paras[0], i)
    qcirc.rz(-paras[1], j)
    qcirc.cx(i, j)

    qcirc.rx(np.pi/2, i)
    qcirc.rz(np.pi/2, j)
    
    State = Statevector(qcirc)

    # print('states2 yahui', State)

    return State

In [347]:
def square_modulus_cost(Paras : list, *args):

    Edge = args[0]
    State = args[1]
    Tauc = args[2]
    N = args[3]

    State_zz = circuit_update_zz(Edge,State, Tauc, N)
    State_theta = circuit_update_theta(Edge,State, Paras, N)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State_theta)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    # print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [348]:
def square_modulus_cost_Yahui(Paras : list, *args):

    Edge = args[0]
    Circ = args[1]
    Tauc = args[2]
    N = args[3]

    State =  Statevector(Circ)

    State_zz = circuit_update_zz(Edge,State, Tauc, N)
    State_theta = circuit_update_theta_Yahui(Edge, Circ, Paras)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State_theta)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [349]:
def square_modulus_cost_light_cone(Paras : list, *args):

    # para_init = np.zeros((len(lightcone_dict[edge]) + 1, 2))

    Edge_list = args[0]
    State = args[1]
    Tauc = args[2]
    Updated_state = args[3]
    N = args[4]

    State_zz = circuit_update_zz(Edge_list[-1], Updated_state, Tauc, N)
    
    for index, edge in enumerate(Edge_list):
        Parameters = [Paras[2*index], Paras[2*index + 1]]
        State = circuit_update_theta(edge,State, Parameters, N)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    # Print the inner product and its square modulus
    # print("Inner product:", inner_product)
    # print("Square modulus of the inner product:", square_modulus)

    return - square_modulus

In [350]:
def warm_start_parameters_lightcone(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list, lightcone_dict: dict):    

    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    # Z term
    for i in range(N):

        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)
        
    ## ZZ term 
    state = Statevector(circ)
    #updated_state = state 
    #updated_state = copy.deepcopy(state)

    print('state in', state)
    #print('updated_state in', updated_state)

    print('\nnumber of columns is:', len(edges_columns))

    for column_index, column in enumerate(edges_columns):
        print('\n##################################################')
        print('column index', column_index, 'column', column)

        if column_index == 0: 

            #first_column_state = state
            first_column_state = copy.deepcopy(state)

            for edge in column:
                
                if len(lightcone_dict[edge]) != 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()
            
                print('\nedge', edge)  

                tauc = tau * edge_coeff_dict[edge]

                para_init = [0,0]

                final = minimize(square_modulus_cost,
                                    para_init,
                                    args = (edge, first_column_state, tauc, N),
                                    jac=False,
                                    bounds=None,
                                    method='L-BFGS-B',
                                    callback=None,
                                    options={'maxiter': 1000})

                para = final.x

                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                edge_params_dict[edge] = para
                # print('edge_params_dict:', edge_params_dict)

                # print('old state', state)
                first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

                print('first column state - new state', first_column_state)
            
        else:
            
            for edge in column: 
                if len(lightcone_dict[edge]) == 0:
                        sys.stderr.write('something is wrong with the lightcones')
                        sys.exit()

                print('\nedge', edge)  

                # print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

                # print('initial state', state)

                updated_state = copy.deepcopy(state)
                #updated_state = state

                print('state in', state, 'column edge', column, edge)
                print('updated_state in', updated_state, 'column edge', column, edge)
                
                for old_edge in lightcone_dict[edge]:
                    # print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
                
                # print('updated_state', updated_state)

                edge_list = lightcone_dict[edge] + [edge]
                # print('edge list', edge_list)
                
                tauc = tau * edge_coeff_dict[edge]

                para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
                # print('para init', para_init)

                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = (edge_list, state, tauc, updated_state, N),
                        jac=False,
                        bounds=None,
                        method='L-BFGS-B',
                        callback=None,
                        options={'maxiter': 1000})

                para = final.x
                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                para = (np.array(para)).reshape(-1, 2)
                # print('param list reshaped', para)
                for index, edge in enumerate(edge_list):
                    edge_params_dict[edge] = para[index] 
                # print('edge_params_dict', edge_params_dict)
            
            # print('\n######### previous column update ########')
            # print('column index', column_index, 'column', column)
            # print('previous column is:', edges_columns[column_index -1])

            for edge in edges_columns[column_index -1]:
                # print('edge', edge, 'parameter', edge_params_dict[edge])
                state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

            # print('circuit updated ad the previous column i.e. column', column_index -1)
            # print('updated statevector', state)

    # print('\n######### last column update ########')
    # print('last column is:', edges_columns[ -1])

    for edge in edges_columns[ -1]:
        # print('edge', edge, 'parameter', edge_params_dict[edge])
        state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    # print('circuit updated at the last column i.e.', len(edges_columns) - 1)
    print('updated statevector', state)

    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    # Concatenate and flatten all arrays into a single array
    flattened_array = np.concatenate(values_as_arrays)
    # Convert the flattened array to a list if needed
    params_list = flattened_array.tolist()
    # print(' params_list' , params_list )

    state = np.array(state)

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss

    return edge_params_dict, params_list, exp_poss_dict

In [351]:
# def warm_start_parameters_lightcone(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list, lightcone_dict: dict):    
    
#     edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer

#     q = QuantumRegister(N, name = 'q')
#     circ = QuantumCircuit(q)
#     circ.clear()
#     circ.h(q[::])

#     tau = 0.2

#     # Z term
#     for i in range(N):

#         #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
#         tauc = tau * edge_coeff_dict[(i,)] 
#         para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
#         edge_params_dict[(i,)] = para
#         circ.ry(para, i)
        
#     ## ZZ term 
#     state = Statevector(circ)
#     updated_state = state 

#     print('\nnumber of columns is:', len(edges_columns))

#     for column_index, column in enumerate(edges_columns):
#         print('\n##################################################')
#         print('column index', column_index, 'column', column)

#         if column_index == 0: 

#             first_column_state = state

#             for edge in column:
                
#                 if len(lightcone_dict[edge]) != 0:
#                     sys.stderr.write('something is wrong with the lightcones')
#                     sys.exit()
            
#                 print('\nedge', edge)  

#                 tauc = tau * edge_coeff_dict[edge]

#                 para_init = [0,0]

#                 final = minimize(square_modulus_cost,
#                                     para_init,
#                                     args = (edge, first_column_state, tauc),
#                                     jac=False,
#                                     bounds=None,
#                                     method='L-BFGS-B',
#                                     callback=None,
#                                     options={'maxiter': 100})

#                 para = final.x

#                 print('opt paramenters', para)
#                 print('final square modulus', final.fun)

#                 edge_params_dict[edge] = para
#                 print('edge_params_dict:', edge_params_dict)

#                 print('old state', state)
#                 first_column_state = circuit_update_theta(edge, first_column_state, para)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

#                 print('first column state - new state', first_column_state)
            
#         else:
            
#             for edge in column: 
#                 if len(lightcone_dict[edge]) == 0:
#                         sys.stderr.write('something is wrong with the lightcones')
#                         sys.exit()

#                 print('\nedge', edge)  

#                 print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

#                 print('initial state', state)

#                 updated_state = copy.deepcopy(state)
                
#                 for old_edge in lightcone_dict[edge]:
#                     print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
#                     updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge])    
                
#                 print('updated_state', updated_state)

#                 edge_list = lightcone_dict[edge] + [edge]
#                 print('edge list', edge_list)
                
#                 tauc = tau * edge_coeff_dict[edge]

#                 para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
#                 print('para init', para_init)

#                 final = minimize(square_modulus_cost_light_cone,
#                         para_init,
#                         args = (edge_list, state, tauc, updated_state),
#                         jac=False,
#                         bounds=None,
#                         method='L-BFGS-B',
#                         callback=None,
#                         options={'maxiter': 100})

#                 para = final.x
#                 print('opt paramenters', para)
#                 print('final square modulus', final.fun)

#                 para = (np.array(para)).reshape(-1, 2)
#                 print('param list reshaped', para)
#                 for index, edge in enumerate(edge_list):
#                     edge_params_dict[edge] = para[index] 
#                 print('edge_params_dict', edge_params_dict)
            
#             print('\n######### previous column update ########')
#             print('column index', column_index, 'column', column)
#             print('previous column is:', edges_columns[column_index -1])

#             for edge in edges_columns[column_index -1]:
#                 print('edge', edge, 'parameter', edge_params_dict[edge])
#                 state = circuit_update_theta(edge, state, edge_params_dict[edge])

#             print('circuit updated ad the previous column i.e. column', column_index -1)
#             print('updated statevector', state)

#     print('\n######### last column update ########')
#     print('last column is:', edges_columns[ -1])

#     for edge in edges_columns[ -1]:
#         print('edge', edge, 'parameter', edge_params_dict[edge])
#         state = circuit_update_theta(edge, state, edge_params_dict[edge])

#     print('circuit updated at the last column i.e.', len(edges_columns) - 1)
#     print('updated statevector', state)

#     #generate params_list
#     values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
#     # Concatenate and flatten all arrays into a single array
#     flattened_array = np.concatenate(values_as_arrays)
#     # Convert the flattened array to a list if needed
#     params_list = flattened_array.tolist()
#     print(' params_list' , params_list )

#     return edge_params_dict, params_list

In [352]:
N = 6
tau = 0.2
r = 10

edges_columns = partition_N(N)
print('edges_columns', edges_columns)
lightcone_dict = find_light_cone(edges_columns)
print('lightcone_dict', lightcone_dict)

#region load qubo instances, get Hamiltonian and edge_coeff_dict
instance_dir = '../instances/complete/N_' + str(N )
with open(instance_dir + '/QUBO_' + str(N ) + 'V_comp_'+ '.gpickle', 'rb') as f:
    G = pickle.load(f)
edge_list = G.edges()
pairs_all = list(itertools.chain.from_iterable(partition_N(N)))
print('pairs_all', pairs_all)
print('edge_list', edge_list)
coeff_list = np.loadtxt(instance_dir + '/QUBO_coeff_' + str(N) + 'V_comp_'+ 'r_'+ str(r)+ '.txt')
h_list = coeff_list[: N ]
J_list = coeff_list[N :]

#extrcact Hamiltonian, edge coefficients and eigen list
H = Hamiltonian_qubo(N, edge_list, h_list, J_list)
eigen_list = H.to_spmatrix().diagonal().real
print('Hamiltonian', H)
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

edges_columns [[(0, 1), (2, 3), (4, 5)], [(0, 3), (2, 5)], [(1, 3), (0, 5), (2, 4)], [(1, 5), (0, 4)], [(3, 5), (1, 4), (0, 2)], [(3, 4), (1, 2)]]
lightcone_dict {(0, 1): [], (2, 3): [], (4, 5): [], (0, 3): [(0, 1), (2, 3)], (2, 5): [(2, 3), (4, 5)], (1, 3): [(0, 3)], (0, 5): [(0, 3), (2, 5)], (2, 4): [(2, 5)], (1, 5): [(1, 3), (0, 5)], (0, 4): [(0, 5), (2, 4)], (3, 5): [(1, 5)], (1, 4): [(1, 5), (0, 4)], (0, 2): [(0, 4)], (3, 4): [(3, 5), (1, 4)], (1, 2): [(1, 4), (0, 2)]}
pairs_all [(0, 1), (2, 3), (4, 5), (0, 3), (2, 5), (1, 3), (0, 5), (2, 4), (1, 5), (0, 4), (3, 5), (1, 4), (0, 2), (3, 4), (1, 2)]
edge_list [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]
Hamiltonian 0.1936 * IIIIIZ
+ 0.5301 * IIIIZI
+ 0.8278 * IIIZII
- 0.8469 * IIZIII
+ 0.1258 * IZIIII
+ 0.9465 * ZIIIII
+ 0.8315 * IIIIZZ
+ 0.6248 * IIIZIZ
+ 0.4479 * IIZIIZ
- 0.6114 * IZIIIZ
+ 0.3099 * ZIIIIZ
- 0.454 * IIIZZI
- 0.7734 * IIZIZI
+ 0.7477 * IZIIZ

In [353]:
# Initialize dictionary for single Pauli Z term with coefficient from h_list
Edge_coeff_dict = {}
Edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(h_list)})
#Initialize dictionary for Pauli ZZ term with coefficient from from J_list
Edge_coeff_dict.update({edge: J_val for edge, J_val in zip(edge_list, J_list)})
print('edge_coeff_dict' , Edge_coeff_dict)
#endregion

edge_coeff_dict {(0,): 0.1936, (1,): 0.5301, (2,): 0.8278, (3,): -0.8469, (4,): 0.1258, (5,): 0.9465, (0, 1): 0.8315, (0, 2): 0.6248, (0, 3): 0.4479, (0, 4): -0.6114, (0, 5): 0.3099, (1, 2): -0.454, (1, 3): -0.7734, (1, 4): 0.7477, (1, 5): 0.2512, (2, 3): 0.5387, (2, 4): -0.5476, (2, 5): 0.4688, (3, 4): -0.945, (3, 5): -0.7465, (4, 5): -0.7301}


In [354]:
edge_params_dict, params_init, exp_poss_dict = warm_start_parameters_lightcone(N, tau, Edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)

state in Statevector([0.07900854+0.j, 0.0853701 +0.j, 0.09767015+0.j,
             0.10553429+0.j, 0.11002152+0.j, 0.11888016+0.j,
             0.13600831+0.j, 0.14695934+0.j, 0.05630571+0.j,
             0.0608393 +0.j, 0.06960497+0.j, 0.07520938+0.j,
             0.07840722+0.j, 0.08472036+0.j, 0.09692679+0.j,
             0.10473108+0.j, 0.08308598+0.j, 0.08977585+0.j,
             0.10271066+0.j, 0.11098066+0.j, 0.11569946+0.j,
             0.12501528+0.j, 0.14302736+0.j, 0.15454355+0.j,
             0.05921151+0.j, 0.06397907+0.j, 0.07319711+0.j,
             0.07909075+0.j, 0.08245362+0.j, 0.08909257+0.j,
             0.10192895+0.j, 0.110136  +0.j, 0.11537134+0.j,
             0.12466074+0.j, 0.14262174+0.j, 0.15410527+0.j,
             0.16065769+0.j, 0.17359343+0.j, 0.19860461+0.j,
             0.21459573+0.j, 0.08221978+0.j, 0.08883991+0.j,
             0.10163988+0.j, 0.10982366+0.j, 0.11449326+0.j,
             0.12371196+0.j, 0.14153626+0.j, 0.15293239+0.j,
             0.

In [355]:
print(exp_poss_dict)
print('\nwarm start fidelity lightcone', list(exp_poss_dict.items())[0])
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

{-5.8098: 0.0685949521246632, -5.182600000000001: 0.07198485647070572, -4.340999999999999: 0.0694772050428278, -3.9320000000000004: 0.040911847856040864, -3.9293999999999993: 0.03724443483354492, -3.6080000000000005: 0.04708770349663269, -3.5771999999999995: 0.02926212471123486, -3.5358: 0.04545241676830128, -3.2038: 0.027448693933975324, -2.8209999999999997: 0.02548152465806824, -2.5102: 0.024197192042902535, -2.500800000000001: 0.02495454048826784, -2.4032: 0.026398923552341764, -2.3644: 0.026047632090898422, -2.2891999999999997: 0.018962020890451278, -2.1632: 0.02713694395069499, -2.1281999999999996: 0.022539783473339157, -2.0981999999999994: 0.013104799605846257, -2.0416: 0.023779530399612007, -2.0023999999999997: 0.016432959223120675, -1.9702: 0.02462742746777931, -1.7275999999999998: 0.016264986799853852, -1.3434: 0.016168313125841733, -1.3012: 0.022786919711949526, -1.1565999999999994: 0.013084771934052537, -1.1016000000000004: 0.022657061682275197, -1.0881999999999994: 0.015459

In [356]:
N = 4
tau = 0.2

edges_columns = partition_N(N)
print('edges_columns', edges_columns)
pairs_all = list(itertools.chain.from_iterable(partition_N(N)))
print('pairs_all', pairs_all)
lightcone_dict = find_light_cone(edges_columns)
print('lightcone_dict', lightcone_dict)

# coefficient dictionary just to test 
edge_coeff_dict = {}

# Set a seed for reproducibility
np.random.seed(127489)
edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(np.random.random(N))})

for keys,values in lightcone_dict.items():
    edge_coeff_dict[keys] = np.random.random()
        
print('edge_coeff_dict' , edge_coeff_dict)

h_list = []
J_list = []

for key,value in edge_coeff_dict.items():
    
    if len(key) == 1:
        h_list.append(value)
    
    else:
        J_list.append(value)

 
print('h_list', h_list)
print('J_list', J_list)

#extrcact Hamiltonian, edge coefficients and eigen list
H = Hamiltonian_qubo(N, pairs_all, h_list, J_list)
eigen_list = H.to_spmatrix().diagonal().real
print('Hamiltonian', H)
print('eigen list', eigen_list[:13])


edges_columns [[(0, 1), (2, 3)], [(0, 3)], [(1, 3), (0, 2)], [(1, 2)]]
pairs_all [(0, 1), (2, 3), (0, 3), (1, 3), (0, 2), (1, 2)]
lightcone_dict {(0, 1): [], (2, 3): [], (0, 3): [(0, 1), (2, 3)], (1, 3): [(0, 3)], (0, 2): [(0, 3)], (1, 2): [(1, 3), (0, 2)]}
edge_coeff_dict {(0,): 0.21398888783253867, (1,): 0.9956435652563932, (2,): 0.15111518356744724, (3,): 0.8852572582785017, (0, 1): 0.3370535623485561, (2, 3): 0.7542588082477806, (0, 3): 0.30816464857975123, (1, 3): 0.7081517529243259, (0, 2): 0.3343009400330923, (1, 2): 0.6990570051258732}
h_list [0.21398888783253867, 0.9956435652563932, 0.15111518356744724, 0.8852572582785017]
J_list [0.3370535623485561, 0.7542588082477806, 0.30816464857975123, 0.7081517529243259, 0.3343009400330923, 0.6990570051258732]
Hamiltonian 0.21398888783253867 * IIIZ
+ 0.9956435652563932 * IIZI
+ 0.15111518356744724 * IZII
+ 0.8852572582785017 * ZIII
+ 0.3370535623485561 * IIZZ
+ 0.7542588082477806 * ZZII
+ 0.30816464857975123 * ZIIZ
+ 0.7081517529243259 *

In [357]:
edge_params_dict, params_list, exp_poss_dict = warm_start_parameters_lightcone(N, tau, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)

state in Statevector([0.14843662+0.j, 0.16170176+0.j, 0.22105588+0.j,
             0.24081068+0.j, 0.15768575+0.j, 0.17177744+0.j,
             0.23482994+0.j, 0.25581567+0.j, 0.21150761+0.j,
             0.23040913+0.j, 0.31498292+0.j, 0.34313157+0.j,
             0.22468671+0.j, 0.24476599+0.j, 0.3346096 +0.j,
             0.3645122 +0.j],
            dims=(2, 2, 2, 2))

number of columns is: 4

##################################################
column index 0 column [(0, 1), (2, 3)]

edge (0, 1)
first column state - new state Statevector([0.13708858+0.j, 0.17150689+0.j, 0.23655112+0.j,
             0.22554833+0.j, 0.14563061+0.j, 0.18219354+0.j,
             0.25129069+0.j, 0.23960231+0.j, 0.19533776+0.j,
             0.24438048+0.j, 0.33706211+0.j, 0.32138422+0.j,
             0.20750932+0.j, 0.25960791+0.j, 0.35806456+0.j,
             0.34140977+0.j],
            dims=(2, 2, 2, 2))

edge (2, 3)
first column state - new state Statevector([0.1139377 +0.j, 0.14254361+0.j, 0.19660347

In [358]:
print(exp_poss_dict)
print('\nwarm start fidelity lightcone', list(exp_poss_dict.items())[0])
print('eigen list', eigen_list[:13])
print('eigen list sorted',  np.sort(eigen_list))

{-2.571878083479452: 0.15494901594549831, -2.378021317354099: 0.13884941372038376, -1.1740560125609307: 0.07661086358715298, -1.1316219873096887: 0.07523460599891187, -1.0790308071353305: 0.07973806875116693, -0.8756540806222136: 0.06398476469276008, -0.7851019648237231: 0.0693625621146647, -0.6360787039332231: 0.07013953564972861, -0.6022556879602254: 0.06774619309835225, -0.0928201591160367: 0.05015563824314249, 0.07532667613354138: 0.046514637306214204, 0.45971542079036587: 0.03709605151429583, 0.8949818223244984: 0.03301010965828698, 1.5095277382458732: 0.02130961307184045, 2.9999755346063837: 0.012117183477994828, 5.38699161219426: 0.0031817431696057356}

warm start fidelity lightcone (-2.571878083479452, 0.15494901594549831)
eigen list [ 5.38699161  2.99997553 -0.09282016 -1.13162199  1.50952774  0.45971542
 -1.17405601 -0.87565408  0.07532668 -1.07903081 -2.57187808 -2.37802132
 -0.78510196]
eigen list sorted [-2.57187808 -2.37802132 -1.17405601 -1.13162199 -1.07903081 -0.875654

In [359]:

edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer

q = QuantumRegister(N, name = 'q')
circ = QuantumCircuit(q)
circ.clear()
circ.h(q[::])

tau = 0.2

# Z term
for i in range(N):

    #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
    tauc = tau * edge_coeff_dict[(i,)] 
    para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)

    edge_params_dict[(i,)] = para
    circ.ry(para, i)
    
## ZZ term 

state = Statevector(circ)
# updated_state = state 

print('number of columns is:', len(edges_columns))

for column_index, column in enumerate(edges_columns):
    print('\n##################################################')
    print('column index', column_index, 'column', column)

    if column_index == 0: 

        first_column_state = state

        for edge in column:
            
            if len(lightcone_dict[edge]) != 0:
                sys.stderr.write('something is wrong with the lightcones')
                sys.exit()
        
            print('\nedge', edge)  

            tauc = tau * edge_coeff_dict[edge]

            para_init = [0,0]

            final = minimize(square_modulus_cost,
                                para_init,
                                args = (edge, first_column_state, tauc),
                                jac=False,
                                bounds=None,
                                method='L-BFGS-B',
                                callback=None,
                                options={'maxiter': 100})

            para = final.x

            print('opt paramenters', para)
            print('final square modulus', final.fun)

            edge_params_dict[edge] = para
            print('edge_params_dict:', edge_params_dict)

            print('old state', state)
            first_column_state = circuit_update_theta(edge, first_column_state, para)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

            print('first column state - new state', first_column_state)
        
    else:
        
        for edge in column: 
            if len(lightcone_dict[edge]) == 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()

            print('\nedge', edge)  

            print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

            print('initial state', state)
            
            updated_state = copy.deepcopy(state)

            for old_edge in lightcone_dict[edge]:
                print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge])    
            
            print('updated_state', updated_state)

            edge_list = lightcone_dict[edge] + [edge]
            print('edge list', edge_list)
            
            tauc = tau * edge_coeff_dict[edge]

            para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
            print('para init', para_init)

            final = minimize(square_modulus_cost_light_cone,
                    para_init,
                    args = (edge_list, state, tauc, updated_state),
                    jac=False,
                    bounds=None,
                    method='L-BFGS-B',
                    callback=None,
                    options={'maxiter': 100})

            para = final.x
            print('opt paramenters', para)
            print('final square modulus', final.fun)

            para = (np.array(para)).reshape(-1, 2)
            print('param list reshaped', para)
            for index, edge in enumerate(edge_list):
                edge_params_dict[edge] = para[index] 
            print('edge_params_dict', edge_params_dict)
        
        print('\n######### previous column update ########')
        print('column index', column_index, 'column', column)
        print('previous column is:', edges_columns[column_index -1])

        for edge in edges_columns[column_index -1]:
            print('edge', edge, 'parameter', edge_params_dict[edge])
            state = circuit_update_theta(edge, state, edge_params_dict[edge])

        print('circuit updated ad the previous column i.e. column', column_index -1)
        print('updated statevector', state)

print('\n######### last column update ########')
print('last column is:', edges_columns[ -1])

for edge in edges_columns[ -1]:
    print('edge', edge, 'parameter', edge_params_dict[edge])
    state = circuit_update_theta(edge, state, edge_params_dict[edge])

print('circuit updated at the last column i.e.', len(edges_columns) - 1)
print('updated statevector', state)

#generate params_list
values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
# Concatenate and flatten all arrays into a single array
flattened_array = np.concatenate(values_as_arrays)
# Convert the flattened array to a list if needed
params_list = flattened_array.tolist()
print(' params_list' , params_list )

number of columns is: 4

##################################################
column index 0 column [(0, 1), (2, 3)]

edge (0, 1)


IndexError: tuple index out of range

In [ ]:
import copy

# Original state
state = {
    'user': {
        'name': 'Alice',
        'preferences': {
            'theme': 'dark',
            'notifications': True
        }
    },
    'session': {
        'id': 'abc123',
        'expires': '2024-12-31T23:59:59Z'
    }
}

# Create a deep copy of the state
updated_state = copy.deepcopy(state)

# Modify the deep copy
updated_state['user']['preferences']['theme'] = 'light'

# Display both states to show they are independent
print("Original state:", state)
print("Updated state:", updated_state)



Original state: {'user': {'name': 'Alice', 'preferences': {'theme': 'dark', 'notifications': True}}, 'session': {'id': 'abc123', 'expires': '2024-12-31T23:59:59Z'}}
Updated state: {'user': {'name': 'Alice', 'preferences': {'theme': 'light', 'notifications': True}}, 'session': {'id': 'abc123', 'expires': '2024-12-31T23:59:59Z'}}


In [ ]:
import copy

# Original state
state = {
    'user': {
        'name': 'Alice',
        'preferences': {
            'theme': 'dark',
            'notifications': True
        }
    },
    'session': {
        'id': 'abc123',
        'expires': '2024-12-31T23:59:59Z'
    }
}

# Create a deep copy of the state
updated_state = state

# Modify the deep copy
updated_state['user']['preferences']['theme'] = 'light'

# Display both states to show they are independent
print("Original state:", state)
print("Updated state:", updated_state)


Original state: {'user': {'name': 'Alice', 'preferences': {'theme': 'light', 'notifications': True}}, 'session': {'id': 'abc123', 'expires': '2024-12-31T23:59:59Z'}}
Updated state: {'user': {'name': 'Alice', 'preferences': {'theme': 'light', 'notifications': True}}, 'session': {'id': 'abc123', 'expires': '2024-12-31T23:59:59Z'}}


In [ ]:
import copy
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Create a Quantum Circuit and initialize the state
circ = QuantumCircuit(2)
circ.h(0)  # Apply Hadamard gate
circ.cx(0, 1)  # Apply CNOT gate
state = Statevector(circ)

# Method 1: Direct Assignment
updated_state1 = state

# Method 2: Deep Copy
updated_state2 = copy.deepcopy(state)

# Print the states
print("Original state data:", state.data)

# Modify updated_state1
updated_state1.data[0] = 0.5

# Modify updated_state2
updated_state2.data[1] = 0.75

# Print the states
print("Original state data:", state.data)
print("Updated state1 data (direct assignment):", updated_state1.data)
print("Updated state2 data (deep copy):", updated_state2.data)

Original state data: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Original state data: [0.5       +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Updated state1 data (direct assignment): [0.5       +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Updated state2 data (deep copy): [0.70710678+0.j 0.75      +0.j 0.        +0.j 0.70710678+0.j]


In [ ]:
import copy
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator

# Create a Quantum Circuit and initialize the state
circ = QuantumCircuit(2)
circ.h(0)  # Apply Hadamard gate
circ.cx(0, 1)  # Apply CNOT gate
state = Statevector(circ)

# Case 1: Direct Assignment
updated_state1 = state

# Create an evolution operator (e.g., another CNOT gate)
evolution_circuit = QuantumCircuit(2)
evolution_circuit.x(0)
evolution_operator = Operator(evolution_circuit)

# Print the states
print("Original state data after direct assignment evolution:", state.data)
print("Updated state1 data (direct assignment):", updated_state1.data)

updated_state1 = updated_state1.evolve(evolution_operator)


# Print the states
print("Original state data after direct assignment evolution:", state.data)
print("Updated state1 data (direct assignment):", updated_state1.data)

# # Case 2: Deep Copy
# updated_state2 = copy.deepcopy(state)
# print("Updated state2 data (deep copy):", updated_state2.data)

# # Evolve updated_state2
# updated_state2 = updated_state2.evolve(evolution_operator)

# # Print the states
# print("Original state data after direct assignment evolution:", state.data)
# print("Updated state1 data (direct assignment):", updated_state1.data)
# print("Updated state2 data (deep copy):", updated_state2.data)

Original state data after direct assignment evolution: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Updated state1 data (direct assignment): [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Original state data after direct assignment evolution: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Updated state1 data (direct assignment): [0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]
